# Setup

In [1]:
import os
from openai import OpenAI
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings #may delete this for line below

#new stuff
from langchain_openai import ChatOpenAI
from langchain.chains import (
    create_history_aware_retriever,
    create_retrieval_chain,
)
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

In [2]:
from dotenv import load_dotenv

# Load the environment file explicitly
from pathlib import Path
dotenv_path = Path('/Users/kyli/Documents/GitHub/lab_gpt/.env')
load_dotenv(dotenv_path=dotenv_path)

# Access variables
openai_api_key = os.getenv("OPENAI_API_KEY")
google_maps_api_key = os.getenv("GOOGLE_MAPS_API_KEY")
huggingface_api_key = os.getenv("HUGGINGFACE_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")
langchain_api_key = os.getenv("LANGCHAIN_API_KEY")

In [3]:
if openai_api_key:
    print('yes')
else:
    print('ugh')

yes


In [55]:
'''
#Create a temporary database with your files
def load_db(file, chain_type, k):
    #Load document - need to load multiple files
    loader = PyPDFLoader(file)
    #Split the documents into pages
    documents = loader.load()
    #Split text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    #Define embedding - turn text chunks into vectors
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    #Create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    #Define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': k})
    #Create chatbot chain. Memory is managed externally
    qa = ConversationalRetrievalChain.from_llm(
        llm = ChatOpenAI(model_name=llm_name, temperature=0),
        chain_type=chain_type,
        retriever=retriever,
        return_source_documents=True,
        return_generated_question=True,
    )
    return qa
    '''

'\n#Create a temporary database with your files\ndef load_db(file, chain_type, k):\n    #Load document - need to load multiple files\n    loader = PyPDFLoader(file)\n    #Split the documents into pages\n    documents = loader.load()\n    #Split text\n    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)\n    docs = text_splitter.split_documents(documents)\n    #Define embedding - turn text chunks into vectors\n    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)\n    #Create vector database from data\n    db = DocArrayInMemorySearch.from_documents(docs, embeddings)\n    #Define retriever\n    retriever = db.as_retriever(search_type="similarity", search_kwargs={\'k\': k})\n    #Create chatbot chain. Memory is managed externally\n    qa = ConversationalRetrievalChain.from_llm(\n        llm = ChatOpenAI(model_name=llm_name, temperature=0),\n        chain_type=chain_type,\n        retriever=retriever,\n        return_source_documents=True,\n    

In [4]:
#Create a temporary database with your files
def load_db_new(file, chain_type, k):
    #Load document - need to load multiple files
    loader = PyPDFLoader(file)
    #Split the documents into pages
    documents = loader.load()
    #Split text
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)
    docs = text_splitter.split_documents(documents)
    #Define embedding - turn text chunks into vectors
    embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key)
    #Create vector database from data
    db = DocArrayInMemorySearch.from_documents(docs, embeddings)
    #Define retriever
    retriever = db.as_retriever(search_type="similarity", search_kwargs={'k': k})
    
    llm = ChatOpenAI()

    #Contextualize question
    contextualize_q_system_prompt = (
        "Given a chat history and the latest user question "
        "which might reference context in the chat history, "
        "formulate a standalone question which can be understood "
        "without the chat history. Do NOT answer the question, just "
        "reformulate it if needed and otherwise return it as is."
    )

    contextualize_q_prompt = ChatPromptTemplate.from_messages(
        [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
        ]
    )
    history_aware_retriever = create_history_aware_retriever(
        llm, retriever, contextualize_q_prompt
    )
    
    # Answer question
    qa_system_prompt = (
        "You are an assistant for question-answering tasks. Use "
        "the following pieces of retrieved context to answer the "
        "question. If you don't know the answer, just say that you "
        "don't know. Use three sentences maximum and keep the answer "
        "concise."
    )

In [57]:
#NEXT TIME YOU CLOSE OUT OF JUPYTER NOTEBOOK ADD DOCARRAY TO CONDA ENV
#!pip install docarray
#!pip install panel

In [5]:
#Create a class using panel and parem that creates an interactive chatbot
#that interacts with the database

import param
#Imports panel library, used to create interactive interfaces (dashboards,
#widgets, etc) GUI
import panel as pn

#the cbfs class inherits from param.Parameterized
class cbfs(param.Parameterized):
    #List to store conversation history
    chat_history = param.List([])
    #String for chatbot's latest response
    answer = param.String("")
    #Latest question sent to document database
    db_query  = param.String("")
    #List of documents/context retrieved from database
    db_response = param.List([])

    #Initialize the class
    def __init__(self,  **params):
        super(cbfs, self).__init__( **params)
        #Empty list to store the chat's visual elements (?)
        self.panels = []
        #Initialize one file for demonstration
        self.loaded_file = "/Users/kyli/Desktop/lab_gpt_example_docs/01_pathophys_tbi.pdf"
        #Calls load_db() method to create a QA omdel
        self.qa = load_db_new(self.loaded_file,"stuff", 4)

    #Loads new file into database
    def call_load_db_new(self, count):
        if count == 0 or file_input.value is None:  # init or no file specified :
            return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")
        else:
            file_input.save("temp.pdf")  # local copy
            #Change the style of the button
            self.loaded_file = file_input.filename
            button_load.button_style="outline"
            self.qa = load_db_new("temp.pdf", "stuff", 4)
            button_load.button_style="solid"
        #Clear chat history
        self.clr_history()
        return pn.pane.Markdown(f"Loaded File: {self.loaded_file}")

    #Process queries
    def convchain(self, query):
        if not query:
            return pn.WidgetBox(pn.Row('User:', pn.pane.Markdown("", width=600)), scroll=True)
        result = self.qa({"question": query, "chat_history": self.chat_history})
        #Store question and answer in chat history
        self.chat_history.extend([(query, result["answer"])])
        #Update database query with new question
        self.db_query = result["generated_question"]
        #Update database with response with new source documents
        self.db_response = result["source_documents"]
        self.answer = result['answer']
        #Add user and chatbot responses to new rows
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=600)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=600, background='#F6F6F6'))
        ])
        inp.value = ''  #clears loading indicator when cleared
        return pn.WidgetBox(*self.panels,scroll=True)

    #Decorator (?)
    @param.depends('db_query ', )
    #Shows latest db query, or message if no queries have been made
    def get_lquest(self):
        if not self.db_query :
            return pn.Column(
                pn.Row(pn.pane.Markdown(f"Last question to DB:", styles={'background-color': '#F6F6F6'})),
                pn.Row(pn.pane.Str("no DB accesses so far"))
            )
        return pn.Column(
            pn.Row(pn.pane.Markdown(f"DB query:", styles={'background-color': '#F6F6F6'})),
            pn.pane.Str(self.db_query )
        )

    @param.depends('db_response', )
    #Shows database lookup results as a list of retrieved docs
    def get_sources(self):
        if not self.db_response:
            return
        rlist=[pn.Row(pn.pane.Markdown(f"Result of DB lookup:", styles={'background-color': '#F6F6F6'}))]
        for doc in self.db_response:
            rlist.append(pn.Row(pn.pane.Str(doc)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    @param.depends('convchain', 'clr_history')
    #Shows conversation history with each exchange in a new row
    def get_chats(self):
        if not self.chat_history:
            return pn.WidgetBox(pn.Row(pn.pane.Str("No History Yet")), width=600, scroll=True)
        rlist=[pn.Row(pn.pane.Markdown(f"Current Chat History variable", styles={'background-color': '#F6F6F6'}))]
        for exchange in self.chat_history:
            rlist.append(pn.Row(pn.pane.Str(exchange)))
        return pn.WidgetBox(*rlist, width=600, scroll=True)

    #Clears chat history
    def clr_history(self,count=0):
        self.chat_history = []
        return


In [7]:
!pip install docarray

In [8]:
cb = cbfs()

file_input = pn.widgets.FileInput(accept='.pdf')
button_load = pn.widgets.Button(name="Load DB", button_type='primary')
button_clearhistory = pn.widgets.Button(name="Clear History", button_type='warning')
button_clearhistory.on_click(cb.clr_history)
inp = pn.widgets.TextInput( placeholder='Enter text here…')

bound_button_load = pn.bind(cb.call_load_db_new, button_load.param.clicks)
conversation = pn.bind(cb.convchain, inp)

jpg_pane = pn.pane.Image( './img/convchain.jpg')

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=300),
    pn.layout.Divider(),
)
tab2= pn.Column(
    pn.panel(cb.get_lquest),
    pn.layout.Divider(),
    pn.panel(cb.get_sources ),
)
tab3= pn.Column(
    pn.panel(cb.get_chats),
    pn.layout.Divider(),
)
tab4=pn.Column(
    pn.Row( file_input, button_load, bound_button_load),
    pn.Row( button_clearhistory, pn.pane.Markdown("Clears chat history. Can use to start a new topic" )),
    pn.layout.Divider(),
    pn.Row(jpg_pane.clone(width=400))
)
dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# ChatWithYourData_Bot')),
    pn.Tabs(('Conversation', tab1), ('Database', tab2), ('Chat History', tab3),('Configure', tab4))
)
pn.extension()
dashboard

ImportError: Could not import docarray python package. Please install it with `pip install docarray`.